# **End-to-End neural ad-hoc ranking with kernel pooling**

Архитектура `K-NRM` (_Kernel Neural Ranking Model_) похожа на `DRMM`, здесь так же учитываются взаимосвязи _каждый-с-каждым_ относительно _слов запроса_ и _документа_. В качестве меры близости используется _cosine similarity_, однако полученная матрица сворачивается с помощью ядер, или кернелов (kernels).
$$
    cosine\ similarity = S_C(A, B) := cos(\theta) = \frac{\bold{A} * \bold{B}} {||\bold{A}||||\bold{B}||}
        = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n}{A_i^2}} \sqrt{\sum_{i=1}^{n}{B_i^2}}}
$$
$$
    M_{i j}=\cos \left(\vec{v}_{t_{i}^{q}}, \vec{v}_{t_{j}^{d}}\right)
$$


Ядро представляет собой `RBF-kernel` (_Radial Basis Function_), который вам может быть знаком из машинного обучения. Каждое такое ядро — это функция от целой строки в матрице взаимодействий $M_{ij}$ между запросом и документом, или, иными словами, взаимодействие одного слова из запроса со всеми словами документа за раз. Это ядро имеет два параметра, которые не обучаются, а фиксируются при инициализации: среднее $\mu_k$ и параметр дисперсии $\sigma_k$, который стоит в знаменателе. Эта сигма обычно одинакова для всех ядер (кернелов).

$$
    K_{k}\left(M_{i}\right)=\sum_{j} \exp \left(-\frac{\left(M_{i j}-\mu_{k}\right)^{2}}{2 \sigma_{k}^{2}}\right)
$$

Давайте разберёмся, как эта формула работает и в чём её смысл. За счёт среднего, параметра $\mu_k$, который уникален для каждого ядра, происходит привязка к конкретному значению в исходной матрице схожести, т.е. к значению _similarity_, с которым сравнивается каждый элемент матрицы $M_{ij}$ попарных взаимодействий эмбеддингов слов запроса и документа. Каждое ядро можно представить как волну, где по оси $OY$ отмечен вес, с которым объект, или токен в документе, входит в конкретную область.

![https://storage.yandexcloud.net/klms-public/production/learning-content/3/10/162/539/3637/image.png](https://storage.yandexcloud.net/klms-public/production/learning-content/3/10/162/539/3637/image.png)

Рассмотрим пример на картинке выше — в нижней части синими точками отмечены документы. Чем левее точка, тем меньше, вплоть до $-1$, cosine similarity между словом в документе, которое представлено точкой, и конкретным зафиксированным словом в запросе. Серая точка вверху символизирует выходное значение конкретного ядра. Видно, что второй кернел имеет наибольшее значение. Это обусловлено тем, что близко к его центру находится целых два объекта, т.е. они наиболее похожи именно на центр волны, которая описывает второе ядро. Далее идёт кернел с номером один, у которого тоже целых две точки, однако они чуть дальше от центра кернела $K1$, нежели такие точки у кернела $K2$. У третьего кернела совсем всё грустно — точки далеки от его центра, а значит, в это ядро практически ничего не попадает и его значение минимально. По сути это сглаженный и дифференцируемый аналог бинов, на которые разбивали отрезок $[-1, 1]$ — ось cosine similarity при расчёте гистограмм.

Для того чтобы симулировать несколько бинов, т.е. оценивать плотность распределения метрики косинусной схожести слов запроса и документа в разных частях интервала, берётся несколько ядер с некоторым шагом между их средним, т.е. $\mu_k$. Допустим, для отрезка $[-1, 1]$ будет $11$ ядер, у первого cреднее в $1$, далее $0.9$, $0.7$ и так далее до $-0.9$. Тогда каждое из ядер даст скалярное числовое значение на выходе, характеризующее схожесть запроса и документа в некоторой окрестности определенной cosine similarity, и объединив все ядра для описания полной картины можно получить вектор. В случае с $11$ ядрами, или кернелами, получаем вектор из $11$ признаков, который несёт в себе ту же информацию, что и Matching Histogram в DRMM. Собственно, такой вектор и получится $\vec{K}\left(M_{i}\right)=\left\{K_{1}\left(M_{i}\right), \ldots, K_{K}\left(M_{i}\right)\right\}$

Ещё раз обращаем внимание на то, что значение ядра тем больше, чем кучнее и ближе к среднему значению $\mu_k$ конкретно этого ядра находятся результирующие косинусные схожести всех слов документа и конкретного слова запроса (аналогия с бинами в DRMM).

Для полноты картины осталось осознать, на что влияет параметр $\sigma_k$, т.е. дисперсия. По сути она задаёт ширину бина, в который попадают слова из документа. На изображении ниже показаны различные параметры $\sigma_k$ для одного кернела со средним, т.е. параметром $\mu_k$, равным $0.7$. Чем больше $\sigma_k$, тем шире бин и тем на большую окрестность обращает внимание этот кернел. Под каждой из $\sigma_k$ справа вынесено изменение метрики MAP (Mean Average Precision) на некотором наборе данных. Это демонстрирует, что подбор $\sigma_k$  для конкретного набора данных в рамках вашей задачи — важная процедура, так как ширина бина влияет на качество модели. Вообще такие ядра можно назвать сглаженной версией TF (Term Frequency), или SOFT-TF, поскольку логика работы с бинами сохраняется и происходит только нестрогое присваивание конкретного слова из документа определённому бину. При этом само слово учитывается в каждом бине, но в большинстве из них с околонулевым весом. По оси $OY$ на графике отражён вес, с которым объект войдёт в сумму: чем дальше от центра ядра, тем меньше вес. Если все объекты находятся далеко от кернела, то, с одной стороны, можно сказать, что этот бин пустой и в методе DRMM было бы записано 0, а с другой — в soft-версии у нас сумма околонулевых значений, т.е. величина незначительная. И наоборот, если в бине или в центре кернела объектов много, то и значение активации на этом ядре будет высоким.

![Демонстрация поведения формы ядра для различных значениях дисперсии](https://storage.yandexcloud.net/klms-public/production/learning-content/3/10/162/539/3637/image_Nd2X53w.png)

Демонстрация поведения формы ядра для различных значениях дисперсии

Когда данная операция применена ко всем строчкам таблицы, на выходе получается $N$ векторов размера $K$, где $K$ — количество ядер. Всё, что осталось сделать — сначала просуммировать по $N$, т.е. по всем словам из запроса, логарифмы значений, полученных на ядрах $\varphi(M)=\sum_{i=1}^{n} \log \vec{K}\left(M_{i}\right)$, таким образом собрав их в единый вектор размера $K$, а затем простым преобразованием вроде $f(q, d)=\tanh \left(w^{T} \varphi(M)+b\right)$ получить выходное значение релевантности. Тут имеем вектор весов $w$ и смещение $b$, это обучаемые параметры.

Легко заметить, что если убрать эмбеддинги из модели, то вообще у модели обучаемых параметров всего 12 (это очень мало): 11 в ядрах и 1 для смещения! Вместо многомиллионных моделей обучается всего 12 параметров. Может показаться странным, но в K-NRM из-за замены жестких бинов на сглаженные версии на кернелы появляется возможность дифференцировать весь алгоритм, считать градиенты и прокидывать их на входные эмбеддинги, по которым оценивается косинусная схожесть. Это даёт удивительную возможность дообучать эмбеддинги слов под свою задачу, т.е. в прямом смысле переопределять значения слов и словосочетаний исходя из потребностей. При этом никто не мешает инициализировать эмбеддинги уже готовыми обученными параметрами, скачанными из интернета, и просто дообучать модель под свои нужды. Это позволяет невероятно быстро обучать модель K-NRM под свои данные, так как практически все веса у нас хорошо инициализированы, и нужно их лишь слегка изменить.

![Архитектура K-NRM](https://storage.yandexcloud.net/klms-public/production/learning-content/3/10/162/539/3637/image_YXBoS8M.png)

Архитектура K-NRM

В практике ML это называется _`Transfer Learning`_, когда знания с одной задачи, в данном случае общеязыкового моделирования, переносятся на другую. В применении, т.е. при инференсе эта модель крайне быстра, потому что тут из операций самое сложное — расчёт матрицы _cosine similarity_ от _каждого-к-каждому_ на входных эмбеддингах. Все остальные операции не требуют затрат и очень просты.

___

Реализуем архитектуру `KNRM`, подготовим данные для обучения и напишем пайплайн тренировки модели.

В качестве датасета будет использоваться набор из пар вопросов с сайта `Quora`, где указано, является ли один из вопросов дубликатом другого. Это задача бинарной классификации. В качестве кандидатов брались максимально похожие вопросы, после чего производилась разметка. В рамках работы по ранжированию будет решаться не задача определения дубликатов, а задача нахождения максимально похожих (т.е. релевантных) вопросов, которые могут удовлетворить пользователя ещё на этапе формулировки. Это можно рассматривать как _suggest_-систему, расположенную под плашкой _"возможно, эти вопросы помогут вам"_. Поэтому введём три уровня релевантности:
* `2` — вопрос является полным дубликатом (согласно оригинальной разметке, это пары с таргетом, равным 1)
* `1` — вопрос очень похож на исходный, однако не является полным дубликатом (согласно оригинальной разметке, это пары с таргетом, равным 0)
* `0` — вопрос не похож на исходный, нерелевантный (таких пар в датасете нет, их можно нагенерировать самостоятельно из общего корпуса всех вопросов)

Этот датасет _Quora Question Pairs_ (`QQP`) входит в набор датасетов GLUE, используемый для всесторонней оценки моделей машинного обучения, связанных с текстом. Скачать его можно [на сайте](https://gluebenchmark.com/tasks/) или непосредственно [по ссылке](https://dl.fbaipublicfiles.com/glue/data/QQP-clean.zip). 

В архиве лежат три файла: 
* `train` используется для тренировки модели;
* `dev` применяется для оценки в рамках домашней работы;
* `test` на данном этапе не используется. 

В качестве текстовых эмбеддингов будут использоваться вектора `GloVe 6B`. Ознакомиться с особенностями этих эмбеддингов вы можете [на официальном сайте](https://nlp.stanford.edu/projects/glove/), а вот точная [ссылка](http://nlp.stanford.edu/data/glove.6B.zip) на скачивание. В архиве представлены вектора разной размерности, однако для проверки работоспособности и оценки решения будут использоваться исключительно вектора размерности 50 (файл glove.6B.50d.txt внутри архива). Понятно, что можно получить более высокое качество (вероятно, незначительно более высокое), используя эмбеддинги размерности 300. Однако это накладывает ограничения на ресурсы (память на диске и ОЗУ), время работы алгоритма и т.д.

## Детали реализации и описание

* `min_token_occurancies` — минимальное количество раз, которое слово (_токен_) должно появиться в выборке, чтобы не быть отброшенным как _низкочастотное_. При значении, равном единице, остаются все слова, которые представлены в датасете.

* `emb_rand_uni_bound` — половина ширины интервала, из которого равномерно (_uniform_) генерируются вектора эмбеддингов (если вектор не представлен в наборе _GloVe_). Если параметр равен $0.2$, то каждая компонента вектора принадлежит $U(−0.2,0.2)$.

* `freeze_knrm_embeddings` — флаг, указывающий на необходимость дообучения эмбеддингов, будут ли по ним считаться градиенты (при `True` дообучение происходить не будет).

* `knrm_kernel_num` — количество ядер в `KNRM`.

* `knrm_out_mlp` — конфигурация _MLP_-слоя на выходе в `KNRM`.

* `dataloader_bs` — размер батча при обучении и валидации модели.

* `train_lr` — _Learning Rate_, использующийся при обучении модели `KNRM`.

* `change_train_loader_ep` — как часто менять/перегенерировать выборку для тренировки модели.

### Блок 1. Реализация токенизации и препроцессинга данных


Необходимо реализовать методы: 

* `handle_punctuation` — очищает строку от пунктуации. Все знаки пунктуации необходимо взять из `string.punctuation`.

* `simple_preproc` — полный препроцессинг строки. Включает в себя обработку пунктуации и приведение к нижнему регистру, а в качестве токенизации (_разбиения предложения на слова или их усеченную версию — токены_) использован метод `nltk.word_tokenize` из библиотеки `nltk`. На выходе — лист со строками (токенами).

* `get_all_tokens` — метод, формирующий список **ВСЕХ** токенов, представленных в подаваемых на вход датасетах (`pd.DataFrame`). Для реализации необходимо сформировать уникальное множество всех текстов, затем рассчитать частотность каждого токена (т.е. после обработки `simple_preproc`) и отсечь те, которые не проходят порог, равный `min_token_occurancies`, с помощью метода `_filter_rare_words`. На выходе — список токенов, для которых будут формироваться эмбеддинги и на которые будут разбиваться оригинальные тексты вопросов.

### Блок 2. Создание матрицы эмбеддингов и словаря токенов


Необходимо реализовать методы: 

* `_read_glove_embeddings` — считывание файла эмбеддингов в словарь, где ключ — это слово, а значение — это вектор эмбеддинга (можно не приводить к `float`-значениям).

* `create_glove_emb_from_file` — метод формирует `matrix` (матрица эмбеддингов размера $N∗D$, где $N$ — количество токенов, $D$ — размерность эмбеддинга), `vocab` (словарь размера $N$, сопоставляющий каждому слову индекс эмбеддинга), `unk_words` — список слов, которые не были в исходных эмбеддингах, и потому для них пришлось генерировать случайный эмбеддинг из равномерного распределения (или другой вектор с заданными характеристиками).

**Обратите внимание:** необходимо в словарь добавить два специальных токена — _`PAD`_ и _`OOV`_, с индексами $0$ и $1$ соответственно. Первый токен используется для заполнения пустот в тензорах (когда один вопрос состоит из бОльшего количества токенов, чем второй, однако их необходимо представить в виде матрицы, в которой строки имеют одинаковую длину) и должен состоять полностью из нулей. Второй токен используется для токенов, которых нет в словаре. К примеру, такое может встретиться в новых текстах: если бы `all_tokens` формировались исключительно по тренировочным данным, то в `dev`-датасете были бы слова, которые модель не видела. Такие слова принято заменять на `OutOfVocab` (_OOV_) токены. Пример вызова этого метода и передаваемых аргументов можете посмотреть в методе `build_knrm_model`. На выходе в матрице эмбеддингов (и в словаре) должны быть как загруженные из файла вектора (для тех слов, которые в нём встретились), так и вектора для новых слов (из `unk_words`, включая `PAD`  и `OOV`). 


> Можете проверить себя на этом этапе: для `min_token_occurancies=1` доля `unk_words` из всех слов должна быть около $30\%$.

### Блок 3. Имплементация `Kernels` и модели `KNRM`


`GaussianKernel` не содержит обучаемых параметров и служит простым нелинейным оператором — необходимо перевести формулу в метод `forward` класса. Параметр $\mu$ отвечает за "среднее" ядра, точку внимания, $\sigma$ — за ширину “бина”.

`KNRM` — класс, характеризующий всю сеть. В нём нужно имплементировать:

* `_get_kernels_layers` — формирует список всех ядер ($K$ штук), применяемых в алгоритме. Важно обратить внимание на автоматическую генерацию $\mu$ для каждого ядра, а также на крайнее правое значение. К примеру, если $K=5$, то $\mu$ должны быть $[-0.75, -0.25, 0.25, 0.75, 1]$, а для $K=11$ $[-0.9, -0.7, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 0.7, 0.9, 1]$. Для понимания принципа генерации проанализируйте симметрию относительно нуля, размер интервалов и краевые значения. Параметр `exact_sigma` означает $\sigma$-значение для крайнего бина, в котором $\mu$ равняется единице.

* `_get_mlp` — формирует выходной _MLP_-слой для ранжирования на основе результата `Kernels`. Точная структура зависит от атрибута `out_layers`. Если `out_layers = []`, то _MLP_ становится линейным слоем из $K$ (признаки равны результатам ядер) в $1$ (финальный скор релевантности). Если `out_layers = [10, 5]`, то архитектура будет следующая: 
    > $K \rightarrow ReLU \rightarrow 10 \rightarrow ReLU \rightarrow 5 \rightarrow ReLU \rightarrow 1$

Обратите внимание, что нелинейность не применяется в конце _MLP_. Таким образом, с помощью цикла нужно научиться в автоматическом режиме генерировать архитектуру выходного слоя. Детальнее о концепции _MLP_ можно узнать в Википедии. В качестве нелинейности, как указано выше, предлагается использовать `ReLU`.

Методы `forward` и `predict` уже реализованы. Обратите внимание, что KNRM будет обучаться в PairWise-режиме. Для этого необходимо соответственно подготовить данные.

**Функции непосредственного расчёта модели:**

* `_get_matching_matrix` — формирует матрицу взаимодействия “каждый-с-каждым” между словами одного и второго вопроса (запрос и документ). В качестве меры используется косинусная схожесть (_cosine similarity_) между эмбеддингами отдельных токенов:
$$
    cosine\ similarity = S_C(A, B) := cos(\theta) = \frac{\bold{A} * \bold{B}} {||\bold{A}||||\bold{B}||}
        = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n}{A_i^2}} \sqrt{\sum_{i=1}^{n}{B_i^2}}}
$$

* `_apply_kernels` — применяет ядра к `matching_matrix`.

### Блок 4. Подготовка Datasets и Dataloaders для обучения и валидации модели


`Dataset` и `Dataloader` — важные части пайплайнов обучения на _PyTorch_. По сути это умные и гибкие обёртки над данными, которые позволяют итерироваться по набору данных. Реализуем два `Dataset` — для _тренировки_ и _валидации_. У них есть существенное отличие — первый работает с триплетами документов (исходный вопрос, вопрос-кандидат1 и вопрос-кандидат2 для обучения в _PairWise_-режиме), второй работает с парами (оценивает отдельно релевантность вопроса-кандидата к исходному вопросу). Метод генерации пар для валидации уже был упомянут выше — `create_val_pairs`.

Для этих датаcетов есть общий класс, который отвечает за обработку текстов — `RankingDataset`.

* `idx_to_text_mapping` отвечает за соотнесение индекса `(id_left и id_right)` с текстом, подробнее в методе `get_idx_to_text_mapping` класса `Solution`.

* `vocab` — маппинг слова в индекс (ведь именно индексы слов подаются в эмбеддинг-слой KNRM в качестве входов, и по ним берётся нужная строка матрицы).

* `oov_val` — значение (индекс) в словаре на случай, если слово не представлено в словаре.

* `preproc_func` — функция обработки и токенизации текста. В примере с созданием `ValPairsDataset` в _init_-методе класса `Solution` видно, что это та же самая функция, которая уже реализована. 

* `max_len` — максимальное количество токенов в тексте.

Итого концептуально `RankingDataset` делает следующее: 

* `__getitem__` возвращает набор признаков для заданной пары или триплета несколько _id_ и _таргет_, где признаки выражены индексами слов в словаре. Сам метод `__getitem__` в классе `RankingDataset` реализовывать не нужно. Однако нужно сделать `_convert_text_idx_to_token_idxs`, который переведёт `id_left/id_right` в индексы токенов в словаре, в частности с помощью функции `_tokenized_text_to_index` (перевод обработанного текста после `preproc_func` в индексы).

Метод `__getitem__` нужно реализовать в наследниках класса — `TrainTripletsDataset` и `ValPairsDataset`, с той лишь разницей, что для тренировки используются тройки документов, а для валидации пары. Сами пары и тройки должны поступать на вход в `index_pairs_or_triplets`. Это список списков _id_ (и конечно, _лейблов_), пример формирования для валидации которого приведён в `create_val_pairs`.

На выходе этого метода ожидается один или два словаря с ключами _query_ и _document_, а также целевая метка (для тренировки — ответ на вопрос, действительно ли первый документ более релевантен запросу, чем второй, для валидации — релевантность от 0 до 2).

Функция `collate_fn` уже написана, её цель — собирать батч из нескольких тренировочных примеров для `KNRM`. Она принимает на вход список из выходов датасетов выше и формирует из них единый `dict` с тензорами в качестве значений. Можете использовать эту функцию в качестве проверки на то, что датасеты реализованы верно. Эта функция должна передаваться, среди прочего, в `DataLoader` с той целью, чтобы автоматически собирать, к примеру, $128$ объектов (триплетов при обучении) в один набор данных, который будет подан в `KNRM` (см. пример в методе `valid`). Более подробно прочитать можно по [ссылке](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn) в документации.

### Блок 5. Тренировка модели 


Теперь осталось соединить всё вместе, чтобы обучать модель!

Реализуйте метод `train`, который совершает $N$ итераций по тренировочному `Dataloader` ($N$ эпох). В зависимости от метода формирования тренировочной выборки при необходимости пересоздавайте выборку каждые `change_train_loader_ep` эпох. Сама тренировочная выборка создаётся в методе `sample_data_for_train_iter`. Вам предлагается самостоятельно придумать и реализовать методику подбора триплетов документов для обучения в _PairWise_-режиме нейросети `KNRM`. В качестве примера можете использовать метод создания валидационного пулла.

Так, авторское решение генерирует порядка $8-10$ тысяч триплетов для обучения. Это недетерминированный процесс (без задания `random_seed`), и потому появляется возможность каждые $K$ эпох менять выборку (не меняя смысла задачи — всё еще нужно определять, релевантнее ли второй документ к данному запросу, чем первый), например — менять _left_ и _right_ _id_. Во время обучения в конце эпохи можете воспользоваться методом `valid` для расчёта $nDCG$. Вам необходимо преодолеть порог в $0.925$. Правильное решение даже без смены набора триплетов за $8-12$ эпох (т.е. за $8-12$ итераций по датасету размера $8-10$ тысяч) с `batch_size` $1024$ (т.е. порядка $100$ обновлений весов) способно преодолеть такой порог (**без дообучения эмбеддингов**). Модель при отправке тренируется с нуля до $20$ эпох, но **не более** $7$ минут.

Домашнее задание можно выполнять сверху вниз по частям (главное закомментировать неработающие куски кода: например, чтобы отрабатывал метод _init_ класса `Solution`), сверяясь с тем, что всё сделано без ошибок. Блок 4 не проверяется отдельно — только вместе с тренировкой модели (так как есть простор и свобода для логики формирования датасета). Корректность работы возвращаемых значений можно проверить локально с помошью `DataLoader` и `collate_fn`.

## Практика

In [1]:
import os

Пути к загруженным данным

In [2]:
GLUE_QQP_DIR = './data/QQP/'
GLOVE_PATH = './data/glove.6B.50d.txt'

### Загрузка данных

In [3]:
if not os.path.exists('data/QQP/'):
    os.system("bash load_data.sh")
else:
    print("QQP data already downloaded successfully!")

QQP data already downloaded successfully!


### Загрузка эмбеддингов

In [4]:
if not os.path.basename(GLOVE_PATH) in os.listdir('./data'):
    os.system("bash load_embeddings.sh")
else:
    print("Embeddings already loaded successfully!")

Embeddings already loaded successfully!


### Реализация

Импорт библиотек

In [5]:
import string
from collections import Counter
from typing import Dict, List, Tuple, Union, Callable, Optional

import nltk
import numpy as np
import math
import pandas as pd
import torch
import torch.nn.functional as F

### `GaussianKernel` class

In [6]:
class GaussianKernel(torch.nn.Module):
    def __init__(self, mu: float = 1.0, sigma: float = 1.0):
        super().__init__()
        self.mu = mu
        self.sigma = sigma
    
    def forward(self, x: torch.Tensor):
        return torch.exp(-(x - self.mu) ** 2 / (2 * self.sigma ** 2))

### `KNRM` class

In [7]:
class KNRM(torch.nn.Module):
    def __init__(
        self,
        embedding_matrix: np.ndarray,
        freeze_embeddings: bool,
        kernel_num: int = 21,
        sigma: float = 0.1,
        exact_sigma: float = 0.001,
        out_layers: List[int] = [10, 5]
    ):
        super().__init__()
        self.embeddings = torch.nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix),
            # Update embeddings in learning process 
            freeze=freeze_embeddings,
            # If specified, the entries at idx is not updated during training
            padding_idx=0
        )
        # K number of kernels
        self.kernel_num = kernel_num
        # Gaussian kernel variance
        self.sigma = sigma
        # Variance of bin, where mu = 1 
        self.exact_sigma = exact_sigma
        # MLP architecture parameter
        self.out_layers = out_layers
        # Kernels layer
        self.kernels = self._get_kernels_layers()
        # MLP layer
        self.mlp = self._get_mlp()
        # Out activation function
        self.out_activation = torch.nn.Sigmoid()


    def _kernels_params(
        self,
        n_kernels: int,
        sigma: float,
        exact_sigma: float,
        lamb: Optional[float]=None,
    ) -> Tuple[List[float], List[float]]:
        list_mus = [1.0]
        list_sigmas = [exact_sigma] # For exact match: small variance => exact match
        if n_kernels == 1:
            return (list_mus, list_sigmas)

        # Bin size
        bin_size = 2.0 / (n_kernels - 1)
        # Add middle of the bin
        list_mus.append(round(1 - bin_size / 2, 5))
        # Different sigmas for different kernels
        if lamb:
            list_sigmas += [bin_size * lamb] * (n_kernels - 1)
        else:
            list_sigmas += [sigma] * (n_kernels - 1)
        for i in range(1, n_kernels - 1):
            mu = round(list_mus[i] - bin_size, 5)
            list_mus.append(mu)
        return (list(reversed(list_mus)), list(reversed(list_sigmas)))
        

    def _get_kernels_layers(self) -> torch.nn.ModuleList:
        kernels = torch.nn.ModuleList()
        mus, sigmas = self._kernels_params(self.kernel_num, self.sigma, self.exact_sigma)
        kernels = []
        for mu, sigma in zip(mus, sigmas):
            kernels.append(GaussianKernel(mu, sigma))

        kernels = torch.nn.ModuleList(kernels)
        return kernels


    def _get_mlp(self) -> torch.nn.Sequential:
        if len(self.out_layers) == 0:
            return torch.nn.Sequential(torch.nn.Linear(self.kernel_num, 1))

        dims = [self.kernel_num, *self.out_layers, 1]
        layers = []
        for i in range(1, len(dims)):
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Linear(dims[i - 1], dims[i]))

        return torch.nn.Sequential(*layers)


    def forward(
            self,
            input_1: Dict[str, torch.Tensor],
            input_2: Dict[str, torch.Tensor]
    ) -> torch.FloatTensor:
        # Paiwise method
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)

        logits_diff = logits_1 - logits_2

        out = self.out_activation(logits_diff)
        return out


    def _get_matching_matrix(self, query: torch.Tensor, doc: torch.Tensor) -> torch.FloatTensor:
        # Broadcast to match query and doc lenght
        query = self.embeddings(query).unsqueeze(2)
        doc = self.embeddings(doc).unsqueeze(1)

        return F.cosine_similarity(query, doc, dim=-1)


    def _apply_kernels(self, matching_matrix: torch.FloatTensor) -> torch.FloatTensor:
        KM = []
        for kernel in self.kernels:
            # shape = [B]
            K = torch.log1p(kernel(matching_matrix).sum(dim=-1)).sum(dim=-1)
            KM.append(K)

        # shape = [B, K]
        kernels_out = torch.stack(KM, dim=1)
        return kernels_out


    def predict(self, inputs: Dict[str, torch.Tensor]) -> torch.FloatTensor:
        # shape = [Batch, Left], [Batch, Right]
        query, doc = inputs['query'], inputs['document']

        # shape = [Batch, Left, Right]
        matching_matrix = self._get_matching_matrix(query, doc)
        # shape = [Batch, Kernels]
        kernels_out = self._apply_kernels(matching_matrix)
        # shape = [Batch]
        out = self.mlp(kernels_out)
        return out

### `Dataset`s classes

In [8]:
class RankingDataset(torch.utils.data.Dataset):
    def __init__(self, index_pairs_or_triplets: List[List[Union[str, float]]],
                 idx_to_text_mapping: Dict[str, str], vocab: Dict[str, int], oov_val: int,
                 preproc_func: Callable, max_len: int = 30):
        self.index_pairs_or_triplets = index_pairs_or_triplets
        self.idx_to_text_mapping = idx_to_text_mapping
        self.vocab = vocab
        self.oov_val = oov_val
        self.preproc_func = preproc_func
        self.max_len = max_len


    def __len__(self):
        return len(self.index_pairs_or_triplets)


    def _tokenized_text_to_index(self, tokenized_text: List[str]) -> List[int]:
        list_of_indexes = []
        for word in tokenized_text:
            list_of_indexes.append(self.vocab.get(word, self.oov_val))
        return list_of_indexes


    def _convert_text_idx_to_token_idxs(self, idx: int) -> List[int]:
        text = self.idx_to_text_mapping[str(idx)]
        text = self.preproc_func(text)
        return self._tokenized_text_to_index(text)


    def __getitem__(self, idx: int):
        pass



class TrainTripletsDataset(RankingDataset):
    def __getitem__(self, idx):
        id_query, id_left, id_right, target = self.index_pairs_or_triplets[idx]
        tokens_query = self._convert_text_idx_to_token_idxs(id_query)
        tokens_left = self._convert_text_idx_to_token_idxs(id_left)
        tokens_right = self._convert_text_idx_to_token_idxs(id_right)
        sample_1 = {'query': tokens_query, 'document': tokens_left}
        sample_2 = {'query': tokens_query, 'document': tokens_right}
        
        return (sample_1, sample_2, target)


class ValPairsDataset(RankingDataset):
    def __getitem__(self, idx):
        id_left, id_right, target = self.index_pairs_or_triplets[idx]
        tokens_left = self._convert_text_idx_to_token_idxs(id_left)
        tokens_right = self._convert_text_idx_to_token_idxs(id_right)
        sample = {'query': tokens_left, 'document': tokens_right}
        
        return (sample, target)


def collate_fn(batch_objs: List[Union[Dict[str, torch.Tensor], torch.FloatTensor]]):
    max_len_q1 = -1
    max_len_d1 = -1
    max_len_q2 = -1
    max_len_d2 = -1

    is_triplets = False
    for elem in batch_objs:
        if len(elem) == 3:
            left_elem, right_elem, label = elem
            is_triplets = True
        else:
            left_elem, label = elem

        max_len_q1 = max(len(left_elem['query']), max_len_q1)
        max_len_d1 = max(len(left_elem['document']), max_len_d1)
        if len(elem) == 3:
            max_len_q2 = max(len(right_elem['query']), max_len_q2)
            max_len_d2 = max(len(right_elem['document']), max_len_d2)

    q1s = []
    d1s = []
    q2s = []
    d2s = []
    labels = []

    for elem in batch_objs:
        if is_triplets:
            left_elem, right_elem, label = elem
        else:
            left_elem, label = elem

        pad_len1 = max_len_q1 - len(left_elem['query'])
        pad_len2 = max_len_d1 - len(left_elem['document'])
        if is_triplets:
            pad_len3 = max_len_q2 - len(right_elem['query'])
            pad_len4 = max_len_d2 - len(right_elem['document'])

        q1s.append(left_elem['query'] + [0] * pad_len1)
        d1s.append(left_elem['document'] + [0] * pad_len2)
        if is_triplets:
            q2s.append(right_elem['query'] + [0] * pad_len3)
            d2s.append(right_elem['document'] + [0] * pad_len4)
        labels.append([label])
    q1s = torch.LongTensor(q1s)
    d1s = torch.LongTensor(d1s)
    if is_triplets:
        q2s = torch.LongTensor(q2s)
        d2s = torch.LongTensor(d2s)
    labels = torch.FloatTensor(labels)

    ret_left = {'query': q1s, 'document': d1s}
    if is_triplets:
        ret_right = {'query': q2s, 'document': d2s}
        return ret_left, ret_right, labels
    else:
        return ret_left, labels

### `Model` class

In [9]:
file_path = GLOVE_PATH


embeddings = {}
with open(file_path, encoding='utf-8') as f:
    for line in f.readlines():
        line = line.split(' ')
        embeddings[line[0]] = line[1:]

In [10]:
len(embeddings), len(set(embeddings)), len(list(embeddings.values())[0])

(400000, 400000, 50)

In [11]:
class Solution:
    def __init__(self, glue_qqp_dir: str, glove_vectors_path: str,
                 min_token_occurancies: int = 1,
                 random_seed: int = 0,
                 emb_rand_uni_bound: float = 0.2,
                 freeze_knrm_embeddings: bool = True,
                 knrm_kernel_num: int = 21,
                 knrm_out_mlp: List[int] = [],
                 dataloader_bs: int = 1024,
                 train_lr: float = 0.001,
                 change_train_loader_ep: int = 10
                 ):
        # Data parameters
        self.glue_qqp_dir = glue_qqp_dir
        self.glove_vectors_path = glove_vectors_path
        self.glue_train_df = self.get_glue_df('train')
        self.glue_dev_df = self.get_glue_df('dev')
        self.dev_pairs_for_ndcg = self.create_val_pairs(self.glue_dev_df)
        # Minimal token occurancies not to be classified as "low-frequency"
        self.min_token_occurancies = min_token_occurancies
        # All
        self.all_tokens = self.get_all_tokens(
            [self.glue_train_df, self.glue_dev_df], self.min_token_occurancies)

        self.random_seed = random_seed
        self.emb_rand_uni_bound = emb_rand_uni_bound
        self.freeze_knrm_embeddings = freeze_knrm_embeddings
        self.knrm_kernel_num = knrm_kernel_num
        self.knrm_out_mlp = knrm_out_mlp
        self.dataloader_bs = dataloader_bs
        self.train_lr = train_lr
        self.change_train_loader_ep = change_train_loader_ep
        # nDCG border
        self.ndcg_border = 0.93

        self.model, self.vocab, self.unk_words = self.build_knrm_model()
        self.idx_to_text_mapping_train = self.get_idx_to_text_mapping(
            self.glue_train_df)
        self.idx_to_text_mapping_dev = self.get_idx_to_text_mapping(
            self.glue_dev_df)

        self.val_dataset = ValPairsDataset(self.dev_pairs_for_ndcg,
              self.idx_to_text_mapping_dev,
              vocab=self.vocab, oov_val=self.vocab['OOV'],
              preproc_func=self.simple_preproc)
        self.val_dataloader = torch.utils.data.DataLoader(
            self.val_dataset, batch_size=self.dataloader_bs, num_workers=0,
            collate_fn=collate_fn, shuffle=False)


    def get_glue_df(self, partition_type: str) -> pd.DataFrame:
        """Load glue DataFrame by `partition type`: ['dev', 'train']
        and transform columns names:
            * qid1 -> id_left;
            * qid2 -> id_right;
            * question1 -> text_left;
            * question2 -> text_right;
            * is_duplicate -> label.

        Parameters
        ----------
        partition_type : `str`
            Which part of data to load: ['dev', 'train'].

        Returns
        -------
        glue_df_fin : `pandas.DataFrame`
            DataFrame with transformed columns names.
        """
        assert partition_type in ['dev', 'train']
        glue_df = pd.read_csv(
            self.glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', dtype=object)
        glue_df = glue_df.dropna(axis=0, how='any').reset_index(drop=True)
        glue_df_fin = pd.DataFrame({
            'id_left'   : glue_df['qid1'],
            'id_right'  : glue_df['qid2'],
            'text_left' : glue_df['question1'],
            'text_right': glue_df['question2'],
            'label'     : glue_df['is_duplicate'].astype(int)
        })
        return glue_df_fin


    def handle_punctuation(self, inp_str: str) -> str:
        """Предобработка:
        Очистка строки от пунктуации.
        
        Parameters
        ----------
        inp_str : str
            Input string to be preprocessed.
        
        Returns
        -------
        inp_str : `str`
            Same string w/o punctuation.
        """
        for p in string.punctuation:
            inp_str = inp_str.replace(p, '')
        return inp_str


    def simple_preproc(self, inp_str: str) -> List[str]:
        """Простая предобработка данных:
        * Очистка от пунктуации;
        * Очистка от пустных строк;
        * Токенизация с помощью `nltk.word_tokenize`
        
        Parameters
        ----------
        inp_str : str
            Input string to be preprocessed.
        
        Returns
        -------
        list_of_tokenized_words : `List[str]`
            List of tokenized words.
        """
        inp_str = self.handle_punctuation(inp_str.lower())
        inp_str = inp_str.strip()
        return nltk.word_tokenize(inp_str)


    def _filter_rare_words(self, vocab: Dict[str, int], min_occurancies: int) -> Dict[str, int]:
        """Filter rare words by `min_occurancies` of word in `vocab` {word: #_word_occurrences}

        Parameters
        ----------
        vocab : `Dict[str, int]`
            Vocabulary of words occurances.
        min_occurancies : `int`
            Minimal token occurancies not to be classified as "low-frequency".

        Returns
        -------
        vocab_filtered : `Dcit[str, int]`
            Filtered vocabulary.
        """
        return {word: cnt for word, cnt in vocab.items() if cnt >= min_occurancies}


    def get_all_tokens(self, list_of_df: List[pd.DataFrame], min_occurancies: int) -> List[str]:
        """Формирует список ВСЕХ токенов, представленных в подаваемых на вход датасетах (`pd.DataFrame`).
        Алгоритм:
            1. Сформировать уникальное множество всех текстов;
            2. Рассчитать частотность каждого токена:
                2.1. Препроцессинг с помощью `simple_preproc`;
                2.2. Отсечь слова согласно порогу, равному `min_token_occurancies` 
                    с помощью метода `_filter_rare_words`. 
            3. Вернуть список токенов, для которых будут формироваться эмбеддинги 
                и на которые будут разбиваться оригинальные тексты вопросов.

        Parameters
        ----------
        list_of_df : `pandas.DataFrame`
            Список датафреймов с данными, из которых будут формироваться токены.
        min_occurancies : `int`
            Минимальное значение частоты появления токена.

        Returns
        -------
        all_tokens : `List[str]`
            Список всех предобработанных и отфильтрованных токенов.
        """
        words_corpus = []
        for df in list_of_df:
            for col in ['text_left', 'text_right']:
                for doc in df[col].values:
                    words_corpus.append(doc)
        words_corpus = ' '.join(set(words_corpus))
        words_corpus = self.simple_preproc(words_corpus)
        words_corpus = Counter(words_corpus)
        words_corpus = self._filter_rare_words(words_corpus, min_occurancies)
        return list(words_corpus)


    def _read_glove_embeddings(self, file_path: str) -> Dict[str, List[str]]:
        """Reads GLOVE Embeddings from `file_path`.

        Parameters
        ----------
        file_path : `str`
            Path to GLOVE Embeddings file.

        Returns
        -------
        embeddings : `Dict[str, List[str]]`
        """
        embeddings = {}
        with open(file_path, encoding='utf-8') as f:
            for line in f.readlines():
                line = line.split(' ')
                embeddings[line[0]] = line[1:]

        return embeddings


    def create_glove_emb_from_file(self,
                                   file_path: str,
                                   inner_keys: List[str],
                                   random_seed: int,
                                   rand_uni_bound: float
            ) -> Tuple[np.ndarray, Dict[str, int], List[str]]:
        """Creates embedding matrix from Glove file 

        Parameters
        ----------
        file_path : `str`
            Path to GLOVE Embeddings file.
        inner_keys : `List[str]`
            Unique list of keys tokens. 
        random_seed : `int`
            Reseed the singleton RandomState instance.
        rand_uni_bound : `float`
            Uniform bounds for OutOfVocabulary word's embedding to be sampled by.

        Returns
        -------
        (emb_matix, vocab, unk_words) : `Tuple[np.ndarray, Dict[str, int], List[str]`

        emb_matrix : `np.ndarray`
            Embedding matrix w/ [N, D] shape, 
                where N - #_of_tokens,
                      D - size of embedding vector.
        vocab : `Dict[str, int]`
            Vocabulary of tokens with 'PAD' and 'OOV' in 0, 1 indexes.
        unk_words : `List[str]`
            List of unknown words.
        """
        np.random.seed(random_seed)

        glove = self._read_glove_embeddings(file_path)
        glove_tokens, inner_tokens = set(glove), set(inner_keys)
        # List of unique unknown words
        unk_words = ['PAD', 'OOV'] + list(inner_tokens.difference(glove_tokens))
        # D = Embeddings size 
        dim = len(list(glove.values())[0])
        # Embedding value sampled from U[+/-rand_uni_bound] for OutOfVocabulary words 
        oov = np.random.uniform(-rand_uni_bound, rand_uni_bound, dim)
        # Create embadding matrix [N, D], N - #_of_tokens
        emb_matrix = [np.zeros(dim), oov]

        vocab = {"PAD": 0, "OOV": 1}
        for i, token in enumerate(inner_tokens, start=2):
            vocab[token] = i
            embedding = glove.get(token, oov)
            # Convert string embedding values from file to float
            embedding = np.array(list(map(float, embedding)))
            emb_matrix.append(embedding)
        # [N, D] embedding matrix to np.ndarray
        emb_matrix = np.array(emb_matrix)
        return emb_matrix, vocab, unk_words


    def build_knrm_model(self) -> Tuple[torch.nn.Module, Dict[str, int], List[str]]:
        """Builds KNRM model.

        Parameters
        ----------
        None

        Returns
        -------
        (knrm, vocab, unk_words) : `Tuple[torch.nn.Module, Dict[str, int], List[str]]`

        knrm : torch.nn.Module
            KNRM model class instance.
        vocab : `Dict[str, int]`
            Vocabulary of tokens with 'PAD' and 'OOV' in 0, 1 indexes.
        unk_words : `List[str]`
            List of unknown words.
        """
        embedding_matrix, vocab, unk_words = self.create_glove_emb_from_file(
            self.glove_vectors_path, self.all_tokens, self.random_seed, self.emb_rand_uni_bound)
        
        torch.manual_seed(self.random_seed)
        knrm = KNRM(embedding_matrix=embedding_matrix,
                    freeze_embeddings=self.freeze_knrm_embeddings,
                    out_layers=self.knrm_out_mlp,
                    kernel_num=self.knrm_kernel_num)
        return knrm, vocab, unk_words


    def _gen_pairs(self,
                   list1: List[Union[str, float]],
                   list2: List[Union[str, float]]):
        return ((x, y) for x in list1 for y in list2)


    def sample_data_for_train_iter(self, inp_df: pd.DataFrame, seed: int) -> List[List[Union[str, float]]]:
        fill_top_to, min_group_size = -1, 2  # todo: vary

        inp_df = inp_df[['id_left', 'id_right', 'label']]
        all_ids = set(inp_df['id_left']).union(set(inp_df['id_right']))

        group_size = inp_df.groupby('id_left').size()
        left_ind_to_use = group_size[group_size >= min_group_size].index.tolist()
        groups = inp_df[inp_df['id_left'].isin(left_ind_to_use)].groupby('id_left')

        np.random.seed(seed)

        out_pairs = []
        for id_left, group in groups:
            ones_ids = group[group.label == 1].id_right.values
            zeroes_ids = group[group.label == 0].id_right.values
            sum_len = len(ones_ids) + len(zeroes_ids)
            num_pad_items = max(0, fill_top_to - sum_len)
            if num_pad_items > 0:
                cur_chosen = set(ones_ids).union(set(zeroes_ids)).union(set(id_left))
                pad_sample = np.random.choice(list(all_ids - cur_chosen), num_pad_items, replace=False).tolist()
            else:
                pad_sample = []
            for doc1, doc2 in self._gen_pairs(ones_ids, zeroes_ids):
                out_pairs.append([id_left, doc1, doc2, 1.0] if np.random.rand() > 0.5 else [id_left, doc2, doc1, 0.0])
            for doc1, doc2 in self._gen_pairs(ones_ids, pad_sample):
                out_pairs.append([id_left, doc1, doc2, 1.0] if np.random.rand() > 0.5 else [id_left, doc2, doc1, 0.0])
            for doc1, doc2 in self._gen_pairs(zeroes_ids, pad_sample):
                out_pairs.append([id_left, doc1, doc2, 1.0] if np.random.rand() > 0.5 else [id_left, doc2, doc1, 0.0])

        return out_pairs

    def create_val_pairs(self,
                         inp_df: pd.DataFrame,
                         fill_top_to: int = 15,
                         min_group_size: int = 2,
                         seed: int = 0) -> List[List[Union[str, float]]]:
        """Creates val pairs for ValPairsDataset.

        Parameters
        ----------
        inp_df : `pandas.DataFrame`
            Input DataFrame.
        fill_top_to : `int`
            Top to fill.
            Default = 15.
        min_group_size : `int`
            Min size of group.
            Default = 2
        seed : `int`
            Random seed.
            Default = 0.

        Returns
        -------
        out_pairs : `List[Union[str, float]]`
            List of pairs to validation.
        """
        # Select columns
        inp_df_select = inp_df[['id_left', 'id_right', 'label']]
        # Size of groups
        inp_df_group_sizes = inp_df_select.groupby('id_left').size()
        # List of ids to use later from `id_left`
        glue_dev_leftids_to_use = list(
            inp_df_group_sizes[inp_df_group_sizes >= min_group_size].index)
        # Groupby by `id_left``
        groups = inp_df_select[inp_df_select.id_left.isin(
            glue_dev_leftids_to_use)].groupby('id_left')
        # Set of all ids from `id_left` and `id_right`
        all_ids = set(inp_df['id_left']).union(set(inp_df['id_right']))
        # List of returned pairs
        out_pairs = []
        # Set RandomSeed
        np.random.seed(seed)

        for id_left, group in groups:
            ones_ids = group[group.label > 0].id_right.values
            zeroes_ids = group[group.label == 0].id_right.values
            sum_len = len(ones_ids) + len(zeroes_ids)
            num_pad_items = max(0, fill_top_to - sum_len)
            if num_pad_items > 0:
                cur_chosen = set(ones_ids).union(
                    set(zeroes_ids)).union({id_left})
                pad_sample = np.random.choice(
                    list(all_ids - cur_chosen), num_pad_items, replace=False).tolist()
            else:
                pad_sample = []
            for i in ones_ids:
                out_pairs.append([id_left, i, 2])
            for i in zeroes_ids:
                out_pairs.append([id_left, i, 1])
            for i in pad_sample:
                out_pairs.append([id_left, i, 0])
        return out_pairs


    def get_idx_to_text_mapping(self, inp_df: pd.DataFrame) -> Dict[str, str]:
        """Creates a mapping between all ids (`id_left` or `id_right`) 
            and text from inputed dataframe.

        Parameters
        ----------
        inp_df : `pandas.DataFrame`
            Input DataFrame.

        Returns
        -------
        left_dict : `Dict[str, str]`
            Dictionary mapping id to text.
        """
        left_dict = (
            inp_df
            [['id_left', 'text_left']]
            .drop_duplicates()
            .set_index('id_left')
            ['text_left']
            .to_dict()
        )
        right_dict = (
            inp_df
            [['id_right', 'text_right']]
            .drop_duplicates()
            .set_index('id_right')
            ['text_right']
            .to_dict()
        )
        left_dict.update(right_dict)
        return left_dict


    def ndcg_k(self,
               y_true: np.ndarray,
               y_pred: np.ndarray,
               gain_scheme: str = 'exp2',
               ndcg_top_k: int = 10) -> float:
        """Metric:
            Calculates the Normalized Discounted Cumulative Gain (NDCG) 
            by k most relevant.

        Parameters
        ----------
        ys_true : `np.ndarray`
            Correct labels rank.
        ys_pred : `np.ndarray`
            Predicted labels rank.
        gain_scheme : `str`
            Gain scheme. Allowed values = ['const', 'exp2']
                * const : gain = rank;
                * exp2  : gain = 2^rank - 1.
            Default = 'exp2'
        ndcg_top_k : `int`
            Top k most relevant objects.
            Default = 10

        Returns
        -------
        ndcg_value : `float`
            Metric value.
        """
        def dcg_k(y_true: np.ndarray,
                  y_pred: np.ndarray,
                  gain_scheme: str,
                  top_k: int)-> float:
            indices = np.argsort(y_pred)[::-1]
            ind = min((len(y_true), top_k))
            y_true_sorted = y_true[indices][:ind]
            gain = 0
            for i, y in enumerate(y_true_sorted, start=1):
                if gain_scheme == 'const':
                    gain += y
                elif gain_scheme == 'exp2':
                    gain += (2 ** y - 1) / math.log2(i + 1)
            return gain

        empirical_dcg = dcg_k(y_true, y_pred, gain_scheme, ndcg_top_k)
        ideal_dcg = dcg_k(y_true, y_true, gain_scheme, ndcg_top_k)
        if ideal_dcg == 0:
            return 0
        else:
            return empirical_dcg / ideal_dcg


    def valid(self, model: torch.nn.Module, val_dataloader: torch.utils.data.DataLoader) -> float:
        labels_and_groups = val_dataloader.dataset.index_pairs_or_triplets
        labels_and_groups = pd.DataFrame(labels_and_groups, columns=['left_id', 'right_id', 'rel'])

        all_preds = []
        for batch in (val_dataloader):
            inp_1, y = batch
            preds = model.predict(inp_1)
            preds_np = preds.detach().numpy()
            all_preds.append(preds_np)
        all_preds = np.concatenate(all_preds, axis=0)
        labels_and_groups['preds'] = all_preds

        ndcgs = []
        for cur_id in labels_and_groups.left_id.unique():
            cur_df = labels_and_groups[labels_and_groups.left_id == cur_id]
            ndcg = self.ndcg_k(cur_df.rel.values.reshape(-1), cur_df.preds.values.reshape(-1))
            if np.isnan(ndcg):
                ndcgs.append(0)
            else:
                ndcgs.append(ndcg)
        return np.mean(ndcgs)


    def _get_train_loader(self, epoch_num: int):
        triplets = self.sample_data_for_train_iter(self.glue_train_df, epoch_num)
        train_dataset = TrainTripletsDataset(triplets,
                                             self.idx_to_text_mapping_train,
                                             vocab=self.vocab,
                                             oov_val=self.vocab['OOV'],
                                             preproc_func=self.simple_preproc)
        train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                                       batch_size=self.dataloader_bs,
                                                       num_workers=0,
                                                       collate_fn=collate_fn,
                                                       shuffle=True)
        return train_dataloader


    def train(self, n_epochs: int):
        opt = torch.optim.SGD(self.model.parameters(), lr=self.train_lr)
        criterion = torch.nn.BCELoss()

        ndcg = 0
        train_dataloader = self._get_train_loader(0)
        for epoch in range(1, n_epochs+1):
            cur_loss = 0
            if epoch % self.change_train_loader_ep == 0:
                train_dataloader = self._get_train_loader(epoch)

            for batch in train_dataloader:
                doc1, doc2, batch_true = batch
                batch_pred = self.model(doc1, doc2)
                loss = criterion(batch_pred, batch_true)
                opt.zero_grad()
                loss.backward()
                opt.step()
                cur_loss += loss.item()

            if epoch % 2 == 0:
                ndcg = self.valid(self.model, self.val_dataloader)

            print(f'\nEpoch: {epoch}/{n_epochs} --- loss: {round(cur_loss, 4)}\tnDCG: {round(ndcg, 4)}')
            if ndcg >= self.ndcg_border:
                print('nDCG Border beated!')
                break


In [12]:
trainer = Solution(glue_qqp_dir=GLUE_QQP_DIR, glove_vectors_path=GLOVE_PATH)
trainer.train(20)


Epoch: 1/20 --- loss: 14.8223	nDCG: 0


NameError: name 'dcg' is not defined

___
___

In [ ]:
class Solution:
    def __init__(self,
                 glue_qqp_dir: str,
                 glove_vectors_path: str,
                 min_token_occurancies: int = 1,
                 random_seed: int = 0,
                 emb_rand_uni_bound: float = 0.2,
                 freeze_knrm_embeddings: bool = True,
                 knrm_kernel_num: int = 21,
                 knrm_out_mlp: List[int] = [],
                 dataloader_bs: int = 1024,
                 train_lr: float = 0.001,
                 change_train_loader_ep: int = 10
                 ):
        self.glue_qqp_dir = glue_qqp_dir
        self.glove_vectors_path = glove_vectors_path
        self.glue_train_df = self.get_glue_df('train')
        self.glue_dev_df = self.get_glue_df('dev')
        self.dev_pairs_for_ndcg = self.create_val_pairs(self.glue_dev_df)
        self.min_token_occurancies = min_token_occurancies
        self.all_tokens = self.get_all_tokens([self.glue_train_df, self.glue_dev_df],
                                              self.min_token_occurancies,
                                              )
        self.random_seed = random_seed
        self.emb_rand_uni_bound = emb_rand_uni_bound
        self.freeze_knrm_embeddings = freeze_knrm_embeddings
        self.knrm_kernel_num = knrm_kernel_num
        self.knrm_out_mlp = knrm_out_mlp
        self.dataloader_bs = dataloader_bs
        self.train_lr = train_lr
        self.change_train_loader_ep = change_train_loader_ep
        # nDCG border
        self.ndcg_border = 0.93

        self.model, self.vocab, self.unk_words = self.build_knrm_model()
        self.idx_to_text_mapping_train = self.get_idx_to_text_mapping(
            self.glue_train_df)
        self.idx_to_text_mapping_dev = self.get_idx_to_text_mapping(
            self.glue_dev_df)

        self.val_dataset = ValPairsDataset(self.dev_pairs_for_ndcg,
                                           self.idx_to_text_mapping_dev,
                                           vocab=self.vocab,
                                           oov_val=self.vocab['OOV'],
                                           preproc_func=self.simple_preproc,
                                           )
        self.val_dataloader = torch.utils.data.DataLoader(self.val_dataset,
                                                          batch_size=self.dataloader_bs,
                                                          num_workers=0,
                                                          collate_fn=collate_fn,
                                                          shuffle=False,
                                                          )

    def get_glue_df(self, partition_type: str) -> pd.DataFrame:
        assert partition_type in ['dev', 'train']
        glue_df = pd.read_csv(self.glue_qqp_dir + f'/{partition_type}.tsv', 
                              header=0, delimiter="\t", 
                              quoting=csv.QUOTE_NONE, encoding='utf-8', 
                              index_col=[0], dtype=object)
        glue_df = glue_df.dropna(axis=0, how='any').reset_index(drop=True)
        glue_df_fin = pd.DataFrame({
            'id_left': glue_df['qid1'],
            'id_right': glue_df['qid2'],
            'text_left': glue_df['question1'],
            'text_right': glue_df['question2'],
            'label': glue_df['is_duplicate'].astype(int)
        })
        return glue_df_fin

    def hadle_punctuation(self, inp_str: str) -> str:
        for punct in string.punctuation:
            inp_str = inp_str.replace(punct, ' ')
        return inp_str

    def simple_preproc(self, inp_str: str) -> List[str]:
        inp_str = self.hadle_punctuation(inp_str.lower())
        inp_str = inp_str.strip()
        return nltk.word_tokenize(inp_str)

    def _filter_rare_words(self, vocab: Dict[str, int], min_occurancies: int) -> Dict[str, int]:
        return {k: c for k, c in vocab.items() if c >= min_occurancies}

    def get_all_tokens(self, list_of_df: List[pd.DataFrame], min_occurancies: int) -> List[str]:
        corpus = []
        for df in list_of_df:
            for col in ['text_left', 'text_right']:
                for doc in df[col].values:
                    corpus.append(doc)
        corpus = ' '.join(set(corpus))
        corpus = self.simple_preproc(corpus)
        corpus = Counter(corpus)
        corpus = self._filter_rare_words(corpus, min_occurancies)
        return list(corpus)

    def _read_glove_embeddings(self, file_path: str) -> Dict[str, List[str]]:
        embed = {}
        with open(file_path, encoding='utf-8') as f:
            for line in f.readlines():
                line = line.split(' ')
                embed[line[0]] = line[1:]

        return embed

    def create_glove_emb_from_file(self, file_path: str, inner_keys: List[str],
                                   random_seed: int, rand_uni_bound: float
                                   ) -> Tuple[np.ndarray, Dict[str, int], List[str]]:
        np.random.seed(random_seed)

        glove = self._read_glove_embeddings(file_path)
        glove_tokens, inner_tokens = set(glove), set(inner_keys)

        unk_words = ['PAD', 'OOV'] + list(inner_tokens.difference(glove_tokens))

        dim = len(list(glove.values())[0])
        oov = np.random.uniform(-rand_uni_bound, rand_uni_bound, dim)
        emb_matrix = [np.zeros(dim), oov]

        vocab = {'PAD': 0, 'OOV': 1}
        for i, token in enumerate(inner_tokens, start=2):
            vocab[token] = i
            vect = glove.get(token, oov)
            vect = np.array(list(map(float, vect)))
            emb_matrix.append(vect)

        emb_matrix = np.array(emb_matrix)
        return emb_matrix, vocab, unk_words

    def build_knrm_model(self) -> Tuple[torch.nn.Module, Dict[str, int], List[str]]:
        emb_matrix, vocab, unk_words = self.create_glove_emb_from_file(self.glove_vectors_path,
                                                                       self.all_tokens,
                                                                       self.random_seed,
                                                                       self.emb_rand_uni_bound,
                                                                       )
        torch.manual_seed(self.random_seed)
        knrm = KNRM(emb_matrix,
                    freeze_embeddings=self.freeze_knrm_embeddings,
                    out_layers=self.knrm_out_mlp,
                    kernel_num=self.knrm_kernel_num,
                    )
        return knrm, vocab, unk_words

    def _gen_pairs(self, list1, list2):
        return ((x, y) for x in list1 for y in list2)

    def sample_data_for_train_iter(self, inp_df: pd.DataFrame, seed: int) -> List[List[Union[str, float]]]:
        fill_top_to, min_group_size = -1, 2  # todo: vary

        inp_df = inp_df[['id_left', 'id_right', 'label']]
        all_ids = set(inp_df['id_left']).union(set(inp_df['id_right']))

        group_size = inp_df.groupby('id_left').size()
        left_ind_to_use = group_size[group_size >= min_group_size].index.tolist()
        groups = inp_df[inp_df['id_left'].isin(left_ind_to_use)].groupby('id_left')

        np.random.seed(seed)

        out_pairs = []
        for id_left, group in groups:
            ones_ids = group[group.label == 1].id_right.values
            zeroes_ids = group[group.label == 0].id_right.values
            sum_len = len(ones_ids) + len(zeroes_ids)
            num_pad_items = max(0, fill_top_to - sum_len)
            if num_pad_items > 0:
                cur_chosen = set(ones_ids).union(set(zeroes_ids)).union(set(id_left))
                pad_sample = np.random.choice(list(all_ids - cur_chosen), num_pad_items, replace=False).tolist()
            else:
                pad_sample = []
            for doc1, doc2 in self._gen_pairs(ones_ids, zeroes_ids):
                out_pairs.append([id_left, doc1, doc2, 1.0] if np.random.rand() > 0.5 else [id_left, doc2, doc1, 0.0])
            for doc1, doc2 in self._gen_pairs(ones_ids, pad_sample):
                out_pairs.append([id_left, doc1, doc2, 1.0] if np.random.rand() > 0.5 else [id_left, doc2, doc1, 0.0])
            for doc1, doc2 in self._gen_pairs(zeroes_ids, pad_sample):
                out_pairs.append([id_left, doc1, doc2, 1.0] if np.random.rand() > 0.5 else [id_left, doc2, doc1, 0.0])

        return out_pairs

    def create_val_pairs(self, inp_df: pd.DataFrame, fill_top_to: int = 15,
                         min_group_size: int = 2, seed: int = 0) -> List[List[Union[str, float]]]:
        inp_df_select = inp_df[['id_left', 'id_right', 'label']]
        inf_df_group_sizes = inp_df_select.groupby('id_left').size()
        glue_dev_leftids_to_use = list(
            inf_df_group_sizes[inf_df_group_sizes >= min_group_size].index)
        groups = inp_df_select[inp_df_select.id_left.isin(
            glue_dev_leftids_to_use)].groupby('id_left')

        all_ids = set(inp_df['id_left']).union(set(inp_df['id_right']))

        out_pairs = []

        np.random.seed(seed)

        for id_left, group in groups:
            ones_ids = group[group.label > 0].id_right.values
            zeroes_ids = group[group.label == 0].id_right.values
            sum_len = len(ones_ids) + len(zeroes_ids)
            num_pad_items = max(0, fill_top_to - sum_len)
            if num_pad_items > 0:
                cur_chosen = set(ones_ids).union(
                    set(zeroes_ids)).union({id_left})
                pad_sample = np.random.choice(
                    list(all_ids - cur_chosen), num_pad_items, replace=False).tolist()
            else:
                pad_sample = []
            for i in ones_ids:
                out_pairs.append([id_left, i, 2])
            for i in zeroes_ids:
                out_pairs.append([id_left, i, 1])
            for i in pad_sample:
                out_pairs.append([id_left, i, 0])
        return out_pairs

    def get_idx_to_text_mapping(self, inp_df: pd.DataFrame) -> Dict[str, str]:
        left_dict = (
            inp_df
            [['id_left', 'text_left']]
            .drop_duplicates()
            .set_index('id_left')
            ['text_left']
            .to_dict()
        )
        right_dict = (
            inp_df
            [['id_right', 'text_right']]
            .drop_duplicates()
            .set_index('id_right')
            ['text_right']
            .to_dict()
        )
        left_dict.update(right_dict)
        return left_dict

    def _dcg_k(self, ys_true: np.array, ys_pred: np.array, top_k: int) -> float:
        indices = np.argsort(ys_pred)[::-1]
        ind = min((len(ys_true), top_k))
        ys_true_sorted = ys_true[indices][:ind]
        gain = 0
        for i, y in enumerate(ys_true_sorted, start=1):
            gain += (2 ** y - 1) / math.log2(i + 1)

        return gain

    def ndcg_k(self, ys_true: np.array, ys_pred: np.array, ndcg_top_k: int = 10) -> float:
        ideal_dcg = self._dcg_k(ys_true, ys_true, ndcg_top_k)
        dcg = self._dcg_k(ys_true, ys_pred, ndcg_top_k)
        if ideal_dcg == 0:
            return 0
        else:
            return dcg / ideal_dcg

    def valid(self, model: torch.nn.Module, val_dataloader: torch.utils.data.DataLoader) -> float:
        labels_and_groups = val_dataloader.dataset.index_pairs_or_triplets
        labels_and_groups = pd.DataFrame(labels_and_groups, columns=['left_id', 'right_id', 'rel'])

        all_preds = []
        for batch in val_dataloader:
            inp_1, y = batch
            preds = model.predict(inp_1)
            preds_np = preds.detach().numpy()
            all_preds.append(preds_np)
        all_preds = np.concatenate(all_preds, axis=0)
        labels_and_groups['preds'] = all_preds

        ndcgs = []
        for cur_id in labels_and_groups.left_id.unique():
            cur_df = labels_and_groups[labels_and_groups.left_id == cur_id]
            ndcg = self.ndcg_k(cur_df.rel.values.reshape(-1), cur_df.preds.values.reshape(-1))
            if np.isnan(ndcg):
                ndcgs.append(0)
            else:
                ndcgs.append(ndcg)

        return np.mean(ndcgs)

    def _get_train_loader(self, epoch_num):
        triplets = self.sample_data_for_train_iter(self.glue_train_df, epoch_num)
        train_dataset = TrainTripletsDataset(triplets,
                                             self.idx_to_text_mapping_train,
                                             vocab=self.vocab,
                                             oov_val=self.vocab['OOV'],
                                             preproc_func=self.simple_preproc,
                                             )
        train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                                       batch_size=self.dataloader_bs,
                                                       num_workers=0,
                                                       collate_fn=collate_fn,
                                                       shuffle=True,
                                                       )
        return train_dataloader

    def train(self, n_epochs: int):
        opt = torch.optim.SGD(self.model.parameters(), lr=self.train_lr)
        criterion = torch.nn.BCELoss()

        ndcg = 0
        train_dataloader = self._get_train_loader(0)
        for epoch in range(1, n_epochs+1):
            cur_loss = 0
            if epoch % self.change_train_loader_ep == 0:
                train_dataloader = self._get_train_loader(epoch)

            for batch in train_dataloader:
                doc1, doc2, batch_true = batch
                batch_pred = self.model(doc1, doc2)
                loss = criterion(batch_pred, batch_true)
                opt.zero_grad()
                loss.backward()
                opt.step()
                cur_loss += loss.item()

            if epoch % 2 == 0:
                ndcg = self.valid(self.model, self.val_dataloader)

            print(f'\nEpoch: {epoch}/{n_epochs} --- loss: {round(cur_loss, 4)}\tnDCG: {round(ndcg, 4)}')
            if ndcg >= self.ndcg_border:
                print('nDCG Border beated!')
                break